In [16]:
# Sketch of possible interface for PT code
# with FEniCS on one side and a Jupyter widget
# using THREE.js/WebGL on the other side.
# Most names here are quickly invented without
# too much thought, so feel free to use other
# names or modify the interface

# Some initial imports
from IPython.display import display
import ipywidgets as widgets
import numpy as np

import pythreejs
from pythreejs import Renderer, Scene, Mesh
from pythreejs import OrthographicCamera, PerspectiveCamera, OrbitControls

In [17]:
# Define mesh and attributes directly
vertices = np.asarray([
    (0.0, 0.0, 0.0),
    (1.0, 0.0, 0.0),
    (1.0, 1.1, 0.0),
    ], dtype="float32")
num_vertices = vertices.shape[0]

triangles = np.asarray([
    (0, 1, 2),
    ], dtype="int32")
num_triangles = triangles.shape[0]

attributes = {}
attributes["f_front"] = np.asarray([0.0, 0.1, 0.2], dtype="float32")
attributes["f_back"]  = np.asarray([0.9, 0.8, 0.7], dtype="float32")
attributes["s_front"] = np.asarray([1.0, 0.5, 0.5], dtype="float32")
attributes["s_back"]  = np.asarray([0.0, 0.5, 0.5], dtype="float32")

uniforms = {
    "time": 0.0,
    "f_min": 0.0,
    "f_max": 1.0,
    #"color": [1.0, 1.0, 1.0],
    }

# TODO: Preintegrated textures

# Define shaders directly
vertexShader = """
//attribute vec4 aColor;
varying vec4 vColor;

void main() {
    //vColor = aColor;
    vColor = vec4(pow(sin(position.x), 2.0), 0.0, 0.0, 1.0);
    gl_Position = projectionMatrix * modelViewMatrix * vec4(position, 1.0);
}
"""

fragmentShader = """
uniform float time;
uniform float f_min;
uniform float f_max;

varying vec4 vColor;

void main(){
    vec2 resolution = vec2(10, 10); // TODO
    vec2 uv = gl_FragCoord.xy / resolution.xy;
//    vec4 tmpPos = texture2D( texturePosition, uv );

    //if (vColor.a < 0.01)
    //    discard;

    vec4 c = vColor.rgba;
    c.r *= sin(time);
    c.g *= f_min;
    c.b *= f_max;

    gl_FragColor = vec4(1.0, sin(uv.x)*sin(uv.y), 0.0, 1.0);
}
"""

In [18]:
# Setup widget using THREE.js and WebGL backend for rendering
from volrenwidget import *
material = VolRenMaterial(
    vertexShader=vertexShader,
    fragmentShader=fragmentShader,
    **uniforms)
geometry = VolRenGeometry(
    vertices = vertices.tobytes(),
    triangles = triangles.tobytes(),
    )

unused = """
material.set_shaders(vertex_shader, fragment_shader)
material.set_uniforms(uniforms)
geometry.set_triangles(coordinates, triangles)
geometry.set_attributes(attributes)
"""

In [19]:
camera_position = [0, 5, 5]
camera_up = [0, 0, 1]

def camera_changed(*args):
    # This is called on interaction, just need to make sense of args
    pass # print("Camera changed:", args)

mesh = Mesh(geometry=geometry, material=material)

scene = Scene(children=[mesh])

camera = PerspectiveCamera(
    position=camera_position,
    up=camera_up)
camera.observe(camera_changed)

controller = OrbitControls(controlling=camera)

renderer = Renderer(camera=camera,
                    scene=scene,
                    renderer_type="webgl",
                    controls=[controller])

In [20]:
display(renderer)

In [8]:
unused = """
# Setup a fixed dolfin function
from dolfin import *
mesh = UnitCubeMesh(1, 1, 1)
V = FunctionSpace(mesh, "Lagrange", 1)
e = Expression("(x[0]-x0) *(x[1]-x1) * (x[2]-x2)",
               x0=0.5, x1=0.5, x2=0.5, degree=1)
f = Function(V)
f.interpolate(e)
"""

unused = """
# Compute sorted Projected Tetrahedra triangles
from dolfinpt import *
pt = ProjectedTetrahedraBuilder()
pt.set_mesh(mesh)
pt.set_function(f)
pt.set_mvp(MVP)


# Get shaders from pt object
vertex_shader = pt.get_vertex_shader();
fragment_shader = pt.get_fragment_shader();

# Get coordinates, triangle indices, attributes from pt object
coordinates = pt.get_coordinates()
triangles = pt.get_triangles()
names = pt.get_attribute_names()
attributes = { pt.get_attribute(name) for name in pt.get_attribute_names() }
uniforms = { pt.get_uniform(name) for name in pt.get_uniform_names() }
"""

unused = """
# Then loop over timesteps and update
# display with new function values!
for t in np.arange(0.0, 1.0, 10):
    e.x0 = 0.5 * t
    f.interpolate(e)
    pt.update_function(f)

    attributes["f_front"][:] = fixme
    attributes["f_back"][:] = fixme

    vrw.resend(["f_front", "f_back"])
"""

In [12]:
from pythreejs import Renderer, Scene, Mesh
from pythreejs import OrthographicCamera, PerspectiveCamera, OrbitControls

from pythreejs import SphereGeometry, LambertMaterial, AmbientLight, DirectionalLight
ball = Mesh(geometry=SphereGeometry(radius=1), 
                                    material=LambertMaterial(color='red'),
                                    position=[2, 1, 0])

scene = Scene(children=[ball, AmbientLight(color='#777777')])

c = PerspectiveCamera(position=[0, 5, 5], up=[0, 0, 1],
                      children=[DirectionalLight(color='white', 
                                                 position=[3, 5, 1], 
                                                 intensity=0.5)])
renderer = Renderer(camera=c, 
                    scene=scene, 
                    controls=[OrbitControls(controlling=c)])
display(renderer)